### Информация
- *Автор*: Зитцер Данил
- *Дата*: среда, 8 октября 2021 г., 10:16:09
- *Описание*: Введение в OpenVINO Toolkit
- *Полезные ссылки*:
    - [OpenVINO Documentation](https://docs.openvinotoolkit.org/latest/index.html)
    - [OpenVINO on GitHub](https://github.com/openvinotoolkit)
    - [OpenVINO Model Zoo](https://github.com/openvinotoolkit/open_model_zoo)
    - [OpenVINO Jupyter Notebooks](https://github.com/openvinotoolkit/openvino_notebooks)

### OpenVINO Toolkit

[Использование OpenVINO Toolkit для развёртывания ускоренных приложений глубокого обучения](https://raw.githubusercontent.com/intel-iot-devkit/smart-video-workshop/master/presentations/OpenVINO_Training_Part1_2021.4.pdf)

### Модель классификации

[mobilenet-v3-small-1.0-224-tf](https://github.com/openvinotoolkit/open_model_zoo/tree/master/models/public/mobilenet-v3-small-1.0-224-tf)

In [1]:
import cv2
import json
import matplotlib.pyplot as plt
import numpy as np

from openvino.inference_engine import IECore
from statistics import mean
from time import time

#### Загрузка сети

In [2]:
ie = IECore()

net = ie.read_network(
    model='./../models/v3-small_224_1.0_float.xml',
    weights='./../models/v3-small_224_1.0_float.bin',
)
exec_net = ie.load_network(network=net, device_name='CPU')

input_key = next(iter(exec_net.input_info))
output_key = next(iter(exec_net.outputs.keys()))

In [3]:
print(input_key)
print(output_key)

input
MobilenetV3/Predictions/Softmax


#### Подготовка словаря объектов ImageNet

In [4]:
imagenet_classes = json.loads(open('./../utils/imagenet_class_index.json').read())
# +1, так как 0 отведён под класс background
imagenet_classes = {int(key) + 1: value[1] for key, value in imagenet_classes.items()}

#### Инференс модели

In [5]:
capture = cv2.VideoCapture('./../videos/sheep.mp4')

success, frame = capture.read()

if not success:
    raise ValueError(f'Can not read first frame')
    
inference_times = []

while True:
    success, frame = capture.read()  # bgr

    if not success:
        break
    
    # подготовка кадра на вход нейронной сети: [Any, Any, 3] --> [1, 3, 224, 224]
    prepared_input = cv2.dnn.blobFromImage(image=frame, size=(224, 224))
    # инференс модели
    tic = time()
    result = exec_net.infer(inputs={input_key: prepared_input})[output_key]
    toc = time()
    inference_times.append(toc - tic)
    
    result_index = result.squeeze().argmax()
    
    cv2.putText(frame, f'object: {imagenet_classes[result_index]}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    cv2.putText(frame, f'fps: {1.0 / mean(inference_times):.0f}', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    cv2.imshow('Press Q for exit', frame)

    key = cv2.waitKey(40) & 0xFF

    if key == ord('q'):
        break

capture.release()
cv2.destroyAllWindows()

### Модель детекции

[person-detection-0200](https://github.com/openvinotoolkit/open_model_zoo/tree/master/models/intel/person-detection-0200)

In [6]:
ie = IECore()

net = ie.read_network(
    model='./../models/person-detection-0200.xml',
    weights='./../models/person-detection-0200.bin',
)
exec_net = ie.load_network(network=net, device_name='CPU')

input_key = next(iter(exec_net.input_info))
output_key = next(iter(exec_net.outputs.keys()))

In [7]:
print(input_key)
print(output_key)

image
detection_out


#### Инференс модели

In [8]:
capture = cv2.VideoCapture('./../videos/pedestrians.mp4')

success, frame = capture.read()

if not success:
    raise ValueError(f'Can not read first frame')
    
inference_times = []

while True:
    success, frame = capture.read()  # bgr

    if not success:
        break
    
    # подготовка кадра на вход нейронной сети: [Any, Any, 3] --> [1, 3, 224, 224]
    prepared_input = cv2.dnn.blobFromImage(image=frame, size=(256, 256))
    # инференс модели
    tic = time()
    result = exec_net.infer(inputs={input_key: prepared_input})[output_key]
    toc = time()
    inference_times.append(toc - tic)
    
    result = result[0, 0, result[0, 0, :, 2] > 0.8]
    
    for image_id, label, confidence, x1, y1, x2, y2 in result:
        x1, x2 = int(x1*frame.shape[1]), int(x2*frame.shape[1])
        y1, y2 = int(y1*frame.shape[0]), int(y2*frame.shape[0])
        
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
        cv2.putText(frame, f'{confidence:.1%}', (x1 + 10, y2 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
    
#     cv2.putText(frame, f'object: {imagenet_classes[result_index]}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    cv2.putText(frame, f'fps: {1.0 / mean(inference_times):.0f}', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    cv2.imshow('Press Q for exit', cv2.resize(frame, (0, 0), fx=0.5, fy=0.5))

    key = cv2.waitKey(40) & 0xFF

    if key == ord('q'):
        break

capture.release()
cv2.destroyAllWindows()